<a href="https://colab.research.google.com/github/celikcedev/qr-code-component/blob/main/lastday_multi_reversal_trend_signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas matplotlib tradingview-screener

In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3

In [ ]:
import yfinance as yf
import talib as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
from tradingview_screener import Query, Column

# PEP8 ile uyumlu hale getirmek için uyarıları engelle
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

def get_symbols() -> list:
    """
    Borsa İstanbul'da işlem gören hisse senetlerinin sembollerini alır.

    Returns:
        list: Sembol isimlerini içeren liste.
    """
    try:
        symbols_tuple = (Query()
                        .select('name')
                        .where(
                            Column('type').isin(['stock'])
                        )
                        .order_by('name', ascending=True)
                        .set_markets('turkey')
                        .limit(750)
                        .get_scanner_data())

        symbols = [f"{symbol}.IS" for symbol in symbols_tuple[1]['name']]
        return symbols
    except Exception as e:
        print(f"Sembol listesi alınırken hata oluştu: {e}")
        return []

def fetch_stock_data(symbol):
    """
    Belirtilen sembol için OHLCV verilerini indirir.

    Args:
        symbol (str): Hisse senedinin sembolü.

    Returns:
        DataFrame or None: İndirilen veriler veya None.
    """
    start_date = datetime.now() - timedelta(days=150)
    end_date = datetime.now()
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        if data.empty:
            raise ValueError("Veri bulunamadı veya eksik.")
        return data
    except Exception as e:
        print(f"{symbol} için veri çekilemedi: {e}")
        return None

def calculate_indicators(data):
    """
    OHLCV verileri üzerinde indikatör hesaplamaları yapar.

    Args:
        data (DataFrame): Hisse senedine ait OHLCV verileri.

    Returns:
        DataFrame: İndikatörler eklenmiş veri.
    """
    try:
        # Verileri ndarray tipine dönüştür
        close = data['Close'].astype(float).values
        high = data['High'].astype(float).values
        low = data['Low'].astype(float).values
        open_ = data['Open'].astype(float).values
        volume = data['Volume'].astype(float).values

        # Bollinger Bandları
        upper_band, middle_band, lower_band = ta.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
        data['Upper_Band'] = upper_band
        data['Middle_Band'] = middle_band
        data['Lower_Band'] = lower_band

        # MACD
        macd, macd_signal, macd_hist = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
        data['MACD'] = macd
        data['MACD_Signal'] = macd_signal

        # MFI
        data['MFI'] = ta.MFI(high, low, close, volume, timeperiod=14)

        # VWAP
        data['VWAP'] = (volume * (high + low + close) / 3).cumsum() / volume.cumsum()

        # Mum Formasyonları
        # Doji, piyasanın kararsızlığını gösterir ve trendin sona erdiğini işaret edebilir.
        data['Doji'] = ta.CDLDOJI(open_, high, low, close)
        # Hammer ise, özellikle düşüş trendi sonunda tersine dönüş sinyali olarak güçlü bir yapı sunar.
        data['Hammer'] = ta.CDLHAMMER(open_, high, low, close)

        # Genellikle düşüş trendinden sonra ortaya çıkar ve yükselişin başlayabileceğine işaret eder.
        data['Morning_Star'] = ta.CDLMORNINGSTAR(open_, high, low, close)
        # Düşüş trendi sonunda görülür ve sonraki hareketin yukarı yönlü olabileceğini gösterir.
        data['Bullish_Engulfing'] = ta.CDLENGULFING(open_, high, low, close)

        return data
    except Exception as e:
        print(f"İndikatör hesaplanırken hata oluştu: {e}")
        return data

def identify_buy_signals(data):
    """
    Al sinyali veren hisseleri tespit eder.

    Args:
        data (DataFrame): İndikatörler eklenmiş hisse verileri.

    Returns:
        dict: Al sinyali veren hisse bilgileri.
    """
    buy_signals = {}
    try:
        latest = data.iloc[-1]

        # Alım koşulları:
        # 1. Fiyat VWAP'ın üzerinde
        # 2. MFI > 50
        # 3. MACD, MACD Signal hattını yukarı kesiyor
        # 4. Doji veya Hammer formasyonu oluşmuş ve hacim artışı var
        price_above_vwap = latest['Close'] > latest['VWAP']
        mfi_condition = latest['MFI'] > 50
        macd_condition = (latest['MACD'] > latest['MACD_Signal']) and (data['MACD'].iloc[-2] <= data['MACD_Signal'].iloc[-2])
        candle_condition = (latest['Doji'] != 0 or latest['Hammer'] != 0) and latest['Volume'] > data['Volume'].mean()

        # Bollinger Band Koşulu: Fiyat alt banda yakınsa veya orta banda geri dönüyorsa
        bollinger_condition = latest['Close'] <= latest['Lower_Band'] or latest['Close'] >= latest['Middle_Band']

        if price_above_vwap and mfi_condition and macd_condition and candle_condition and bollinger_condition:
            # Hangi formasyonun sinyal verdiğini belirle
            if latest['Doji'] != 0:
                formation = 'Doji'
            elif latest['Hammer'] != 0:
                formation = 'Hammer'
            elif latest['Morning_Star'] != 0:
                formation = 'Morning Star'
            elif latest['Bullish_Engulfing'] != 0:
                formation = 'Bullish Engulfing'
            else:
                formation = 'Mum deseni bulunamadı.'

            buy_signals = {
                'Close': latest['Close'],
                'VWAP': latest['VWAP'],
                'MFI': latest['MFI'],
                'MACD': latest['MACD'],
                'MACD_Signal': latest['MACD_Signal'],
                'Bollinger_Position': 'Near Lower Band' if latest['Close'] <= latest['Lower_Band'] else 'Near Middle Band',
                'Candle_Formation': formation
            }
    except Exception as e:
        print(f"Al sinyali tespit edilirken hata oluştu: {e}")

    return buy_signals

def plot_indicators(symbol, data, buy=False):
    """
    Hisse senedi için indikatörleri ve al/sat sinyallerini grafiğe döker.

    Args:
        symbol (str): Hisse senedinin sembolü.
        data (DataFrame): İndikatörler eklenmiş hisse verileri.
        buy (bool): Al sinyali var mı.
    """
    try:
        plt.figure(figsize=(14, 10))

        # Fiyat Grafiği ve VWAP
        ax1 = plt.subplot(3, 1, 1)
        ax1.plot(data.index, data['Close'], label='Close Price')
        ax1.plot(data.index, data['VWAP'], label='VWAP', linestyle='--')
        ax1.set_title(f"{symbol} - Fiyat ve VWAP")
        ax1.legend()
        ax1.grid(True)

        if buy:
            ax1.scatter(data.index[-1], data['Close'][-1], marker='^', color='green', label='Al Sinyali')
            ax1.legend()

        # Bollinger Bandları
        ax2 = plt.subplot(3, 1, 2, sharex=ax1)
        ax2.plot(data.index, data['Close'], label='Close Price')
        ax2.plot(data.index, data['Upper_Band'], label='Upper Bollinger Band', linestyle='--', color='orange')
        ax2.plot(data.index, data['Middle_Band'], label='Middle Bollinger Band', linestyle='-', color='grey')
        ax2.plot(data.index, data['Lower_Band'], label='Lower Bollinger Band', linestyle='--', color='orange')
        ax2.set_title("Bollinger Bandları")
        ax2.legend()
        ax2.grid(True)

        # MACD
        ax3 = plt.subplot(3, 1, 3, sharex=ax1)
        ax3.plot(data.index, data['MACD'], label='MACD', color='blue')
        ax3.plot(data.index, data['MACD_Signal'], label='MACD Signal', color='red')
        ax3.bar(data.index, data['MACD'] - data['MACD_Signal'], label='MACD Histogram', color='grey')
        ax3.set_title("MACD")
        ax3.legend()
        ax3.grid(True)

        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Grafik çizilirken hata oluştu: {e}")

def main():
    """
    Ana fonksiyon. Stratejiyi çalıştırır.
    """
    # Hisse senedi sembollerini al
    symbols = get_symbols()
    if not symbols:
        print("Sembol listesi boş. Program sonlandırılıyor.")
        return

    buy_list = []

    # Her sembol için veri indir, indikatörleri hesapla ve al sinyali kontrol et
    for symbol in symbols:
        print(f"İşleniyor: {symbol}")
        stock_data = fetch_stock_data(symbol)
        if stock_data is not None:
            stock_data = calculate_indicators(stock_data)
            buy_signal = identify_buy_signals(stock_data)
            if buy_signal:
                buy_list.append((symbol, buy_signal))
                # Sinyal bilgilerini formatlı şekilde yazdır
                print(f"\nAl Sinyali: {symbol}")
                print(f"Close Price: {buy_signal['Close']}")
                print(f"VWAP: {buy_signal['VWAP']:.2f}")
                print(f"MFI: {buy_signal['MFI']:.2f}")
                print(f"MACD: {buy_signal['MACD']:.2f}")
                print(f"MACD Signal: {buy_signal['MACD_Signal']:.2f}")
                print(f"Mum Formasyonu: {buy_signal['Candle_Formation']}\n")

    # Al sinyali veren hisse sayısını yazdır
    print(f"\nToplam Al Sinyali Veren Hisse Sayısı: {len(buy_list)}\n")

    # Al sinyali veren hisseler için grafik çiz
    for symbol, signal in buy_list:
        stock_data = fetch_stock_data(symbol)
        if stock_data is not None:
            stock_data = calculate_indicators(stock_data)
            plot_indicators(symbol, stock_data, buy=True)

if __name__ == "__main__":
    main()